<a href="https://colab.research.google.com/github/Wegjl99/Pythonista/blob/main/earthquakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Earthquake Analysis: Large quakes preceded by ≥3 M6+ events within 3 days
import requests
import pandas as pd
from datetime import datetime, timedelta

# Step 1: Fetch earthquakes ≥ M6 since 2000 from USGS API
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": "2000-01-01",
    "endtime": "2025-12-01",
    "minmagnitude": 6,
    "orderby": "time"
}

print("Fetching data from USGS...")
response = requests.get(url, params=params)
data = response.json()

# Step 2: Convert to DataFrame
records = []
for feature in data['features']:
    props = feature['properties']
    records.append({
        "time": datetime.utcfromtimestamp(props['time']/1000),
        "mag": props['mag'],
        "place": props['place']
    })

df = pd.DataFrame(records)
df.sort_values("time", inplace=True)

# Step 3: Identify large quakes (M ≥ 7.5)
large_quakes = df[df['mag'] >= 7.5]

# Step 4: For each large quake, check preceding 3 days for ≥3 M6+ quakes
results = []
for _, row in large_quakes.iterrows():
    window_start = row['time'] - timedelta(days=3)
    window_end = row['time']
    preceding = df[(df['time'] >= window_start) & (df['time'] < window_end) & (df['mag'] >= 6)]
    if len(preceding) >= 3:
        results.append({
            "mainshock_time": row['time'],
            "mainshock_mag": row['mag'],
            "mainshock_place": row['place'],
            "preceding_count": len(preceding),
            "preceding_quakes": preceding.to_dict('records')
        })

# Output the count and details
print(f"\nNumber of large earthquakes preceded by ≥3 M6 quakes in 3 days: {len(results)}")
for r in results:
    print("\nMainshock:", r['mainshock_time'], r['mainshock_mag'], r['mainshock_place'])
    print("Preceding quakes count:", r['preceding_count'])

# Optional: Save results to CSV
pd.DataFrame(results).to_csv("earthquake_analysis_results.csv", index=False)
print("\nResults saved to earthquake_analysis_results.csv")

Fetching data from USGS...


/tmp/ipython-input-3581726661.py:25: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "time": datetime.utcfromtimestamp(props['time']/1000),



Number of large earthquakes preceded by ≥3 M6 quakes in 3 days: 28

Mainshock: 2000-11-16 07:42:16.930000 7.8 135 km SE of Kokopo, Papua New Guinea
Preceding quakes count: 3

Mainshock: 2000-11-17 21:01:56.490000 7.8 138 km SSW of Kokopo, Papua New Guinea
Preceding quakes count: 5

Mainshock: 2002-09-08 18:44:23.710000 7.6 68 km ESE of Aitape, Papua New Guinea
Preceding quakes count: 3

Mainshock: 2002-11-03 22:12:41.518000 7.9 2002 Denali Fault, Alaska Earthquake
Preceding quakes count: 3

Mainshock: 2004-11-11 21:26:41.150000 7.5 63 km NW of Maubara, Timor Leste
Preceding quakes count: 3

Mainshock: 2006-05-03 15:26:40.290000 8.0 2006 Tonga Earthquake
Preceding quakes count: 3

Mainshock: 2006-11-15 11:14:13.570000 8.3 2006 Kuril Islands Earthquake
Preceding quakes count: 4

Mainshock: 2007-09-28 13:38:57.880000 7.5 Volcano Islands, Japan region
Preceding quakes count: 5

Mainshock: 2009-09-30 10:16:09.250000 7.6 30 km WSW of Pariaman, Indonesia
Preceding quakes count: 3

Mainshock: